In [1]:
FUNCTION read_dap_evi,url,sites
; open the DAP file
 evi_file = ncdf_open(url)
; get the latitude and longitude resolutions from the global attributes
 ncdf_attget,evi_file,"geospatial_lat_resolution",lat_res,/global
 ncdf_attget,evi_file,"geospatial_lon_resolution",lon_res,/global
; get the time variable
 time_id = ncdf_varid(evi_file,"time")
 ncdf_varget,evi_file,time_id,time
 nrecs = n_elements(time)
; and the time variable units attribute
 ncdf_attget,evi_file,time_id,"units",value
 time_units = string(value)
; get the year, month and day from the time units string
 result = strsplit(time_units," ",/extract)
 ymd = strsplit(result[2],"-",/extract)
; get the hour, minute and seconds
 hms = strsplit(result[3],":",/extract)
; get the Julian date from the netCDF time
 jul_time = time + JulDay(fix(ymd[1]),fix(ymd[2]),fix(ymd[0]),0,0,0)
; ... and get the year, month, day etc from the Julian date
 CalDat, jul_time, month, day, year, hour, minute, second
;print, year[-1], month[-1], day[-1], hour[-1], minute[-1], second[-1]
; get the latitude and longitude variables from the netCDF file
 lat_id = ncdf_varid(evi_file,"latitude")
 ncdf_varget,evi_file,lat_id,latitude
 lon_id = ncdf_varid(evi_file,"longitude")
 ncdf_varget,evi_file,lon_id,longitude
; NOTE: evi is dimensioned as [longitude,latitude,time]
;                          eg [19160,14902,365]
 nsites = n_elements(sites.name)
 for i=0,nsites-1 do begin
  print,"Processing site: ",sites.name[i]
; get the latitude and longitude indices
   lat_index = fix(((latitude[0]-sites.latitude[i])/lat_res)+0.5)
   if sites.longitude[i]<0 then sites.longitude[i] = float(360) + sites.longitude[i]
   lon_index = fix(((sites.longitude[i]-longitude[0])/lon_res)+0.5)
; get the offset and count for the data subset
   offset = [lon_index-1,lat_index-1,0]
   count = [3,3,nrecs]
; and now get the EVI data
   evi_id = ncdf_varid(evi_file,"evi")
   ncdf_varget,evi_file,evi_id,evi,offset=offset,count=count
; create the data structure
   new = {name:sites.name[i],year:year,month:month,day:day,$
          hour:hour,minute:minute,second:second,$
          evi:evi}
; add this structure to the array
   if i eq 0 then data = replicate(new,nsites) else data[i] = new
 endfor
; close the netCDF file
 ncdf_close,evi_file
; return the data structure
 return,data
END

In [2]:
sites = {name:["Calperum","Sturt Plains","Whroo"],$
         latitude:[-34.00206,-17.15090,-36.67305],$
         longitude:[140.58912,133.35055,145.02621]}

In [3]:
; define the DAP URL and the file name
dap_path = "http://www.auscover.org.au/thredds/dodsC/auscover/lpdaac-aggregates/c5/v2-nc4/aust/"
evi_file = "MOD13Q1.005/MOD13Q1.aggregated.aust.005.enhanced_vegetation_index.ncml"
; get the full URL and open the netCDF file
evi_url = dap_path+evi_file

In [4]:
data = read_dap_evi(evi_url,sites)

Processing site: Calperum
Processing site: Sturt Plains
Processing site: Whroo


In [5]:
for i=0,n_elements(data)-1 do begin
 nrecs = n_elements(data[i].year)
; get the decimal year for plotting
 julian_date = JulDay(data[i].month,data[i].day,data[i].year,$
                      data[i].hour,data[i].minute,data[i].second)
 doy = julian_date - JulDay(1,1,data[i].year)
 diy = intarr(nrecs)
 diy = 365
 idx = where((data[i].year mod 4) eq 0,count)
 diy[idx] = 366
 ydy = data[i].year + doy/diy + data[i].hour/(24*diy) + data[i].minute/(24*60*diy)
 window,i
 plot,ydy,data[i].evi[1,1,*],max_value=1.0,min_value=-0.2,$
      xrange=[2000,2017],xtickinterval=1,$
      xtitle="Years",ytitle="EVI",$
      title=sites.name[i]
endfor

In [ ]:
evi_file = ncdf_open(evi_url)
ncdf_attget,evi_file,"geospatial_lat_resolution",lat_res,/global
ncdf_attget,evi_file,"geospatial_lon_resolution",lon_res,/global
time_id = ncdf_varid(evi_file,"time")
ncdf_varget,evi_file,time_id,time
nrecs = n_elements(time)
ncdf_attget,evi_file,time_id,"units",value
time_units = string(value)
result = strsplit(time_units," ",/extract)
ymd = strsplit(result[2],"-",/extract)
hms = strsplit(result[3],":",/extract)
jul_time = time + JulDay(fix(ymd[1]),fix(ymd[2]),fix(ymd[0]),0,0,0)
CalDat, jul_time, month, day, year, hour, minute, second
lat_id = ncdf_varid(evi_file,"latitude")
ncdf_varget,evi_file,lat_id,latitude
lon_id = ncdf_varid(evi_file,"longitude")
ncdf_varget,evi_file,lon_id,longitude
nsites = n_elements(sites.name)

In [ ]:
for i=0,nsites-1 do begin
  print,"Processing site: ",sites.name[i]
; get the latitude and longitude indices
  lat_index = fix(((latitude[0]-sites.latitude[i])/lat_res)+0.5)
  if sites.longitude[i]<0 then sites.longitude[i] = float(360) + sites.longitude[i]
  lon_index = fix(((sites.longitude[i]-longitude[0])/lon_res)+0.5)
; get the offset and count for the data subset
  offset = [lon_index-1,lat_index-1,0]
  count = [3,3,nrecs]
; and now get the EVI data
  evi_id = ncdf_varid(evi_file,"evi")
  ncdf_varget,evi_file,evi_id,evi,offset=offset,count=count
; create the data structure
  new = {name:sites.name[i],year:year,month:month,day:day,$
         hour:hour,minute:minute,second:second,$
         evi:evi}
; add this structure to the array
  if i eq 0 then data = replicate(new,nsites) else data[i] = new
endfor
; close the netCDF file
ncdf_close,evi_file